In [39]:
from pysmt.shortcuts import *
import pysmt.typing as types
import numpy as np

n = 16     # precisão
a = 4
b = 27

def bv_sel(z,i):                    # seleciona o bit i do BitVec "z"
    return BVExtract(z,start=i,end=i)

def declare(i): #declara um bitvector de tamanho n
    state = {}
    #declaram-se os BV com margem de manobra para operações (invariante)
    state['x'] = Symbol('x'+str(i),types.BVType(2*n)) 
    state['y'] = Symbol('y'+str(i),types.BVType(2*n))
    state['z'] = Symbol('z'+str(i),types.BVType(2*n))
    state['p'] = Symbol('p'+str(i),INT) #-1 - erro; 0 - loop; 1 - par; 2 - ímpar; 3 - final
    return state

def init(state):
    return And(Equals(state['z'],BVZero(2*n)), Equals(state['p'], Int(0)), BVULT(state['x'], BV(2**n, 2*n)), BVULT(state['y'], BV(2**n, 2*n)))

def init_ab(state,a,b):
    return And(Equals(state['z'],BVZero(2*n)), Equals(state['p'], Int(0)), Equals(state['x'], BV(a,2*n)), Equals(state['y'], BV(b,2*n)))

In [40]:
def trans(curr, prox):
    tend = And(Equals(curr['p'], Int(0)), Equals(prox['p'], Int(3)), Equals(curr['y'], BVZero(2*n)),
               Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    
    tendl = And(Equals(curr['p'], Int(3)), Equals(prox['p'], Int(3)), 
                  Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    
    todd = And(Equals(curr['p'], Int(0)), Equals(prox['p'], Int(2)), Equals(bv_sel(curr['y'],0), BVOne(1)),
               Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    
    toddof = And(Equals(curr['p'], Int(2)), Equals(prox['p'], Int(-1)), curr['x'] > BVSub(BV(2**n-1,2*n), curr['z']),
                 Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
                 
    toddt = And(Equals(curr['p'], Int(2)), Equals(prox['p'], Int(0)), Equals(prox['y'], curr['y'] - BVZExt(BVOne(1), 2*n-1)), 
                Equals(prox['z'], curr['z'] + curr['x']), Equals(prox['x'], curr['x']), Not(curr['x'] > BVSub(BV(2**n-1,2*n), curr['z'])))
                 
    teven = And(Equals(curr['p'], Int(0)), Equals(prox['p'], Int(1)), Not(Equals(curr['y'], BVZero(2*n))), Equals(bv_sel(curr['y'],0), BVZero(1)),
                Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
                 
    tevenof = And(Equals(curr['p'], Int(1)), Equals(prox['p'], Int(-1)), Equals(bv_sel(curr['x'], n-1), BVOne(1)),
                  Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
                 
    tevent = And(Equals(curr['p'], Int(1)), Equals(prox['p'], Int(0)), Equals(prox['x'], BVLShl(curr['x'], BVZExt(BVOne(1), 2*n-1))),
                 Equals(prox['y'], BVLShr(curr['y'], BVZExt(BVOne(1), 2*n-1))), Equals(curr['z'], prox['z']), Not(Equals(bv_sel(curr['x'], n-1), BVOne(1))))
    
    tofl = And(Equals(curr['p'], Int(-1)), Equals(prox['p'], Int(-1)), 
               Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    
    return Or(tend, tendl, todd, toddof, toddt, teven, tevenof, tevent, tofl)

In [41]:
def gera_tracok(declare,init,trans,k):
    with Solver(name="z3") as s:
        trace = [declare(i) for i in range(k)]
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i],trace[i+1]))
        if s.solve():
            for i in range(k-1):
                print(f'(%s,%s,%s,%s)' % (s.get_value(trace[i]['p']), s.get_value(trace[i]['x']),
                                        s.get_value(trace[i]['y']),  s.get_value(trace[i]['z'])), end = " ")
            print(f'(%s,%s,%s,%s)' % (s.get_value(trace[i]['p']), s.get_value(trace[i]['x']),
                                        s.get_value(trace[i]['y']),  s.get_value(trace[i]['z'])))

gera_tracok(declare,init,trans,25)

(0,1279_32,65_32,0_32) (2,1279_32,65_32,0_32) (0,1279_32,64_32,1279_32) (1,1279_32,64_32,1279_32) (0,2558_32,32_32,1279_32) (1,2558_32,32_32,1279_32) (0,5116_32,16_32,1279_32) (1,5116_32,16_32,1279_32) (0,10232_32,8_32,1279_32) (1,10232_32,8_32,1279_32) (0,20464_32,4_32,1279_32) (1,20464_32,4_32,1279_32) (0,40928_32,2_32,1279_32) (1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32) (-1,40928_32,2_32,1279_32)


In [44]:
def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        with Solver(name="z3") as s:
            trace = [declare(i) for i in range(k)]
            a = np.random.randint(2**n)
            b = np.random.randint(2**n)
            print(a,b)
            s.add_assertion(init_ab(trace[0],a,b))
            for i in range(k-1):
                s.add_assertion(trans(trace[i],trace[i+1]))
                s.add_assertion(Not(inv(trace[i], a, b)))
            if s.solve():
                for i in range(k-1):
                    print(f'(%s,%s,%s,%s)' % (s.get_value(trace[i]['p']), s.get_value(trace[i]['x']),
                                        s.get_value(trace[i]['y']),  s.get_value(trace[i]['z'])), end = " ")
                print(f'(%s,%s,%s,%s)' % (s.get_value(trace[k-1]['p']), s.get_value(trace[k-1]['x']),
                                        s.get_value(trace[k-1]['y']),  s.get_value(trace[k-1]['z'])))
                print('Failed!')
                return
            else:
                print('Conseguiu!')

def inv(state,a,b):
    print(BV(a*b,2*n))
    print(a*b)
    return Equals(BVAdd(BVMul(state['x'], state['y']), state['z']), BV(a*b, 2*n))
                
bmc_always(declare,init,trans,inv,25)

10899 26188
(0,10899_32,26188_32,0_32)
Failed!


In [27]:
#ele está a desfazer tudo em função do último que calcula. Em vez de conformar o trace[i] em função do trace[i-1], ele está a redefinir trace[i-1] para se encaixar no trace[i]
#preciso de me lembrar de cada momento, de alguma forma (dicionário dos resultados no trace)
def gera_traco_iterativo(declare,init,trans):
    with Solver(name="z3") as s:
        last = declare(0)
        s.push()
        s.add_assertion(init(last))
        if not s.solve():
            print("what?!")
            return
        s.pop()
        trace = {}
        trace[0] = {'p':s.get_value(last['p']), 'x':s.get_value(last['x']), 'y':s.get_value(last['y']), 'z':s.get_value(last['z'])}
        print(f'(%s,%s,%s,%s)->' % (s.get_value(trace[0]['p']), s.get_value(trace[0]['x']),
                                    s.get_value(trace[0]['y']),  s.get_value(trace[0]['z'])), end = "")
        i = 0
        while s.get_value(trace[i]['p']).constant_value() != 3 and s.get_value(trace[i]['p']).constant_value() != -1:
            i += 1
            last = declare(i)
            s.push()
            s.add_assertion(trans(trace[i-1], last))
            if not s.solve():
                print("what?!") 
                return
            s.pop()
            trace[i] = {'p':s.get_value(last['p']), 'x':s.get_value(last['x']), 'y':s.get_value(last['y']), 'z':s.get_value(last['z'])}
            print(f'(%s,%s,%s,%s)->' % (s.get_value(trace[i]['p']), s.get_value(trace[i]['x']),
                                        s.get_value(trace[i]['y']),  s.get_value(trace[i]['z'])), end = "")
        print("")
        for state in trace:
            print(f'(%s,%s,%s,%s)->' % (s.get_value(trace[state]['p']), s.get_value(trace[state]['x']),
                                        s.get_value(trace[state]['y']),  s.get_value(trace[state]['z'])), end = "")
            
gera_traco_iterativo(declare,init,trans)

(0,0_32,0_32,0_32)->(3,0_32,0_32,0_32)->
(0,0_32,0_32,0_32)->(3,0_32,0_32,0_32)->

In [26]:
def gera_traco(declare,init,trans):
    with Solver(name="z3") as s:
        trace = []
        trace.append(declare(0)) 
        s.add_assertion(init(trace[0]))
        if not s.solve():
            print("what?!")
            return
        i = 0
        while s.get_value(trace[i]['p']).constant_value() != 3 and s.get_value(trace[i]['p']).constant_value() != -1:
            i += 1
            last = declare(i)
            s.add_assertion(trans(trace[i-1], last))
            if not s.solve():
                print("what?!")
                return
            trace.append(last)
        for state in trace:
            print(f'(%s,%s,%s,%s)->' % (s.get_value(state['p']), s.get_value(state['x']),
                                        s.get_value(state['y']),  s.get_value(state['z'])), end = "")
gera_traco(declare,init,trans)

(0,65535_32,65535_32,0_32)->(2,0_32,1_32,0_32)->(0,0_32,8192_32,0_32)->(1,0_32,4_32,0_32)->(-1,34784_32,4_32,0_32)->
(0,17392_32,8_32,0_32)->(1,17392_32,8_32,0_32)->(0,34784_32,4_32,0_32)->(1,34784_32,4_32,0_32)->(-1,34784_32,4_32,0_32)->